In [ ]:
import pandas as pd
import re

def process_dutgraph_log(file_path):
  """
  Processes a dutgraph.log file and extracts relevant information.

  Args:
    file_path: Path to the dutgraph.log file.

  Returns:
    A pandas DataFrame containing the extracted data.
  """

  with open(file_path, 'r') as f:
    lines = f.readlines()

  # Extract Zone and Slot information
  zone_slot_match = re.search(r'Zone (\d+), Slot (\d+)', ''.join(lines))
  zone = int(zone_slot_match.group(1))
  slot = int(zone_slot_match.group(2))

  # Extract product name
  product_match = re.search(r'Mapping source: .+\\(.*?)\\', ''.join(lines))
  product = product_match.group(1)

  # Read the data into a pandas DataFrame
  df = pd.read_csv(file_path, sep=',', comment=';')

  # Extract relevant columns for each socket
  sockets_data = []
  for i in range(8):
    socket_data = df[[
        f'COOLDUTY.{i}',
        f'ES0_TEMP.{i}',
        f'HTR_DUTY.{i}',
        f'SNK_TEMP.{i}',
    ]]
    socket_data['socket'] = i
    sockets_data.append(socket_data)

  # Concatenate the data for all sockets
  df_sockets = pd.concat(sockets_data)

  # Add Zone, Slot, and product information
  df_sockets['Zone'] = zone
  df_sockets['Slot'] = slot
  df_sockets['Product'] = product

  return df_sockets

# Example usage:
file_path = 'dutgraph.log'  # Replace with your file path
df_result = process_dutgraph_log(file_path)
print(df_result)